## Import Libraries

In [1]:
import numpy as np
from numpy.linalg import inv
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import auxiliary as aux
%matplotlib inline
plt.rcParams["figure.figsize"] = (10, 6)

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [2]:
class Line():
    def __init__(self):
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None

In [30]:
# Number of fits to average on
N = 5
gamma = 0.3

DEBUG_MODE = True

curvature_tolerance = 1000
x_dist_tolerance = (3.35, 3.7)
paralell_tolerance = 0.3

def curve_check(left_line, right_line):
    global curvature_tolerance
    curv_diff = abs(left_line.radius_of_curvature - right_line.radius_of_curvature)
#     print("Curve =", left_line.radius_of_curvature, right_line.radius_of_curvature, curv_diff)
    out = curv_diff < curvature_tolerance
#     print("curve_check =",out)
    return curv_diff < curvature_tolerance


def center_check(left_line, right_line):
    global x_dist_tolerance
    line_dist = abs(left_line.line_base_pos) + abs(right_line.line_base_pos)
#     print("Line Dist =", line_dist)
    out = x_dist_tolerance[0] <= line_dist <= x_dist_tolerance[1]
#     print("center_check =",out)
    return out


def paralell_check(left_line, right_line):
    global paralell_tolerance
    mean_dist = np.mean(np.abs(left_line.bestx - right_line.bestx))* aux.pixe2meters['x']
    line_dist = abs(left_line.line_base_pos) + abs(right_line.line_base_pos)
#     print("Mean Dist =", mean_dist)
    out = abs(mean_dist - line_dist) < paralell_tolerance
#     print("paralell_check =",out)
    return out


def sanity_check(left_line, right_line):
    return curve_check(left_line, right_line) and center_check(left_line, right_line) and paralell_check(left_line, right_line)


def drawing(image, binary, M, left_line, right_line, ploty):
    global DEBUG_MODE
    
#     print("\n\nLeft:", left_line.bestx)
#     print("Right:", right_line.bestx)
    
    drawed = aux.back_to_original(image, binary, inv(M), left_line.bestx, right_line.bestx, ploty)
    
    if DEBUG_MODE:
        font = cv2.FONT_HERSHEY_SIMPLEX
        text = 'Radius Curvature = {}(m)'.format(round(left_line.radius_of_curvature,2))
        cv2.putText(drawed,text,(10,50), font, 1.5,(255,255,255),2,cv2.LINE_AA)

        text = 'Left curve to center = {}(m)'.format(round(left_line.line_base_pos,2))
        cv2.putText(drawed,text,(10,100), font, 1.5,(255,255,255),2,cv2.LINE_AA)

        text = 'Right curve to center = {}(m)'.format(round(right_line.line_base_pos, 2))
        cv2.putText(drawed,text,(10,150), font, 1.5,(255,255,255),2,cv2.LINE_AA)
        
        diff = right_line.line_base_pos + left_line.line_base_pos
        if diff < 0:
            side = 'right'
        else:
            side = 'left'
        
        text = 'Vehicle is {}m {} of center'.format(round(abs(diff), 2), side)
        cv2.putText(drawed,text,(10,200), font, 1.5,(255,255,255),2,cv2.LINE_AA)
    
    return drawed


def reset(binary):
    # Apply Sliding Window
    left_fit, left_fitx, right_fit, right_fitx, ploty = aux.sliding_window_polyfit(binary, plot=False)   
    
    # Left Line
    left_line = Line()
    left_line.recent_xfitted = left_fitx
    left_line.current_fit = left_fit
    left_line.radius_of_curvature = aux.curvature(aux.vehicle_center[1], left_fitx, ploty)
    left_line.line_base_pos = aux.distance_center(left_fitx)
    
    left_line.allx, left_line.ally = ([left_fitx], [ploty])
    left_line.best_fit = left_fit
    left_line.bestx = left_fitx
    

    # Right Line
    right_line = Line()
    right_line.recent_xfitted = right_fitx
    right_line.current_fit = right_fit
    right_line.radius_of_curvature = aux.curvature(aux.vehicle_center[1], right_fitx, ploty)
    right_line.line_base_pos = aux.distance_center(right_fitx)
    
    right_line.allx, right_line.ally = ([right_fitx], [ploty])
    right_line.best_fit = right_fit
    right_line.bestx = right_fitx
    
    return left_line, right_line, ploty

scoop = None

def smoothing(line, ploty):
    global N, gamma, scoop
    
    if len(line.allx) > 0:
        # Get the points from the past N frames
        pastx = np.concatenate(line.allx[-N:])
        pasty = np.concatenate(line.ally[-N:])

        # Get averaged fit
        avg_fit = np.polyfit(pasty, pastx, 2)

        # Update Best Fit
        line.best_fit = line.current_fit * gamma + (1 - gamma) * avg_fit

        # Get best X
        line.bestx = line.best_fit[0]*ploty**2 + line.best_fit[1]*ploty + line.best_fit[2]
    else:
        line.best_fit = line.current_fit
        line.bestx = line.bestx
        
    line.allx.append(line.bestx)
    line.ally.append(ploty)
        

def look_ahead(binary, left_line, right_line):
    left_fit, left_fitx, right_fit, right_fitx, ploty = aux.update_polyfit(binary, left_line.best_fit, right_line.best_fit)
    
    # Left Line
    left_line.recent_xfitted = left_fitx
    left_line.current_fit = left_fit
    left_line.radius_of_curvature = aux.curvature(aux.vehicle_center[1], left_fitx, ploty)
    left_line.line_base_pos = aux.distance_center(left_fitx)

    # Right Line
    right_line.recent_xfitted = right_fitx
    right_line.current_fit = right_fit
    right_line.radius_of_curvature = aux.curvature(aux.vehicle_center[1], right_fitx, ploty)
    right_line.line_base_pos = aux.distance_center(right_fitx)
    
    return left_line, right_line, ploty

In [31]:
left_line, right_line, ploty = (None, None, None)
reset_threshold = 3
count_before_reset = reset_threshold

def final_pipeline(raw_image):
    global left_line, right_line, ploty, count_before_reset, reset_threshold
    # 1 - Undistort Image
    image = aux.undistort(raw_image)
    # 2 - Warp Image
    M, warped = aux.bird_eye(image)
    # 3 - Binnary Warped Image
    binary = aux.binary_image(warped)
    
    if count_before_reset == reset_threshold:
        left_line, right_line, ploty = reset(binary)
        count_before_reset = 0
    else:
        left_line, right_line, ploty = look_ahead(binary, left_line, right_line)
        
        # If pass the sanity check append to list of points
        if sanity_check(left_line, right_line):
#             print('* Sanity PASS')

            # Apply smoothing
            smoothing(left_line, ploty)
            smoothing(right_line, ploty)
            count_before_reset = 0

        else:
#             print('* Sanity FAIL')
            count_before_reset += 1
    
    marked_image = drawing(image, binary, M, left_line, right_line, ploty)
    
    return marked_image

### Function to warp images
---
Time to put the warping process inside a function.

In [32]:
white_output = 'test2.mp4'

# clip1 = VideoFileClip("project_video.mp4").subclip(22,27)

clip1 = VideoFileClip("./Test_Videos/project_video.mp4")

# clip1 = VideoFileClip("project_video.mp4").subclip(0,2)

# clip1 = VideoFileClip("project_video.mp4")

white_clip = clip1.fl_image(final_pipeline) #NOTE: this function expects color images!!

%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test2.mp4
[MoviePy] Writing video test2.mp4



 12%|█▏        | 152/1261 [01:43<12:34,  1.47it/s]


 24%|██▍       | 303/1261 [03:26<10:52,  1.47it/s]


 36%|███▌      | 454/1261 [05:09<09:11,  1.46it/s]


 48%|████▊     | 605/1261 [06:52<07:29,  1.46it/s]


...

 72%|███████▏  | 907/1261 [10:17<04:00,  1.47it/s]


 84%|████████▍ | 1057/1261 [11:59<02:18,  1.47it/s]


 96%|█████████▌| 1205/1261 [13:40<00:38,  1.46it/s]


100%|█████████▉| 1260/1261 [14:17<00:00,  1.47it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test2.mp4 

CPU times: user 15min 32s, sys: 2min 12s, total: 17min 45s
Wall time: 14min 18s
